In [1]:
import re
import string
import pickle
import keras
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPool2D, Reshape
from keras.layers import Input, concatenate, Dropout
from keras.layers import Embedding, Concatenate
from keras.optimizers import Adam, SGD, RMSprop
from keras import optimizers
from keras import regularizers
from keras.utils.vis_utils import plot_model
from nltk.corpus import stopwords


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary

In [27]:
train = pd.read_csv('./train_en.tsv', delimiter='\t',encoding='utf-8')
dev = pd.read_csv('./dev_en.tsv', delimiter='\t',encoding='utf-8')

In [28]:
def clean_tweets(tweet):
    tweet = re.sub('@(\\w{1,15})\b', '', tweet)
    tweet = tweet.replace("via ", "")
    tweet = tweet.replace("RT ", "")
    tweet = tweet.lower()
    return tweet
    
def clean_url(tweet):
    tweet = re.sub('http\\S+', '', tweet, flags=re.MULTILINE)   
    return tweet
    
def remove_stop_words(tweet):
    stops = set(stopwords.words("english"))
    stops.update(['.',',','"',"'",'?',':',';','(',')','[',']','{','}'])
    toks = [tok for tok in tweet if not tok in stops and len(tok) >= 3]
    return toks
    
def stemming_tweets(tweet):
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in tweet]
    return stemmed_words

def remove_number(tweet):
    newTweet = re.sub('\\d+', '', tweet)
    return newTweet

def remove_hashtags(tweet):
    result = ''

    for word in tweet.split():
        if word.startswith('#') or word.startswith('@'):
            result += word[1:]
            result += ' '
        else:
            result += word
            result += ' '

    return result

In [29]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aniket.joshi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
def preprocessing(tweet, swords = True, url = True, stemming = True, ctweets = True, number = True, hashtag = True):

    if ctweets:
        tweet = clean_tweets(tweet)

    if url:
        tweet = clean_url(tweet)

    if hashtag:
        tweet = remove_hashtags(tweet)
    
    twtk = TweetTokenizer(strip_handles=True, reduce_len=True)

    if number:
        tweet = remove_number(tweet)
    
    tokens = [w.lower() for w in twtk.tokenize(tweet) if w != "" and w is not None]

    if swords:
        tokens = remove_stop_words(tokens)

    if stemming:
        tokens = stemming_tweets(tokens)

    text = " ".join(tokens)

    return text

In [31]:
train_text  = train['text'].map(lambda x: preprocessing(x, swords = False, url = True, stemming =False, ctweets = True, number = False, hashtag = False))
y_train  = train['HS']
id_train = train['id']

test_text  = dev['text'].map(lambda x: preprocessing(x, swords = False, url = True, stemming = False, ctweets = True, number = False, hashtag = False))
y_test  = dev['HS']
id_test = dev['id']

data = np.concatenate((train_text, test_text), axis=0)
classes = np.concatenate((y_train, y_test), axis=0)


In [32]:
(y_train == 1).sum()

3783

In [33]:
from keras.utils import np_utils

In [34]:
def word_embeddings(word_index, num_words, word_embedding_dim):
    embeddings_index = {}
    
    f = open('./glove.6B.300d.txt', 'r', encoding='utf-8')
    
    for line in tqdm(f):
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    
    f.close()

    matrix = np.zeros((num_words, word_embedding_dim))
    
    for word, i in word_index.items():
        if i >= max_features:
            continue
        
        embedding_vector = embeddings_index.get(word)
        
        if embedding_vector is not None:
            matrix[i] = embedding_vector

    return matrix

In [35]:
embedding_dim = 300
max_features = 25000
maxlen = 100
batch_size = 32
epochs = 5
filter_sizes = [3,4]
num_filters = 512
drop = 0.5

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data)

X = tokenizer.texts_to_sequences(train_text)
Y = tokenizer.texts_to_sequences(test_text)

tweets = sequence.pad_sequences(X, maxlen=maxlen)
x_test = sequence.pad_sequences(Y, maxlen=maxlen)

word_index = tokenizer.word_index

num_words = min(max_features, len(word_index) + 1)

In [36]:
x_train, x_val, y_train, y_val = train_test_split(tweets, y_train, test_size=0.1, random_state=None)


In [37]:
from tqdm import tqdm
embedding_matrix = word_embeddings(word_index, num_words, embedding_dim)

400000it [00:27, 14666.93it/s]


In [38]:
tweet_input = Input(shape=(maxlen,), dtype='int32')

embedding = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=True)(tweet_input)

reshape = Reshape((maxlen, embedding_dim, 1))(embedding)

cnn1 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max1 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(cnn1)

cnn2 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='tanh')(reshape)

max2 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(cnn2)


concatenated_tensor = Concatenate(axis=1)([max1, max2])

flatten = Flatten()(concatenated_tensor)

dropout = Dropout(drop)(flatten)

dens = Dense(num_filters, activation='relu')(dropout)

output = Dense(1, activation='sigmoid')(dens)

model = Model(inputs=tweet_input, outputs=output)

opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.01)

model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 100, 300)     5505600     input_4[0][0]                    
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 100, 300, 1)  0           embedding_4[0][0]                
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 98, 1, 512)   461312      reshape_2[0][0]                  
__________________________________________________________________________________________________
conv2d_4 (

In [39]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
callbacks_list = [early_stopping,mcp_save]
model.fit(x_train, y_train, batch_size=batch_size, epochs=20, shuffle=True, callbacks = callbacks_list,validation_data=(x_val, y_val))

y_pred = (model.predict(x_test, batch_size=batch_size) > .5).astype(int)

Train on 8100 samples, validate on 900 samples
Epoch 1/20
8100/8100 [==============================] - 3s 409us/step - loss: 0.9243 - acc: 0.6428 - val_loss: 0.5916 - val_acc: 0.6944
Epoch 2/20
8100/8100 [==============================] - 3s 361us/step - loss: 0.4829 - acc: 0.7714 - val_loss: 0.7651 - val_acc: 0.6044
Epoch 3/20
8100/8100 [==============================] - 3s 363us/step - loss: 0.4032 - acc: 0.8209 - val_loss: 0.5708 - val_acc: 0.7167
Epoch 4/20
8100/8100 [==============================] - 3s 361us/step - loss: 0.3457 - acc: 0.8502 - val_loss: 0.6028 - val_acc: 0.7167
Epoch 5/20
8100/8100 [==============================] - 3s 363us/step - loss: 0.3094 - acc: 0.8672 - val_loss: 0.5943 - val_acc: 0.7256
Epoch 6/20
8100/8100 [==============================] - 3s 362us/step - loss: 0.2722 - acc: 0.8915 - val_loss: 0.5956 - val_acc: 0.7400
Epoch 7/20
8100/8100 [==============================] - 3s 364us/step - loss: 0.2415 - acc: 0.9046 - val_loss: 0.6404 - val_acc: 0.7189
E

In [40]:
from sklearn.metrics import f1_score, precision_score, accuracy_score, recall_score

print("F1.........: %f" %(f1_score(y_test, y_pred, average="macro")))
print("Precision..: %f" %(precision_score(y_test, y_pred, average="macro")))
print("Recall.....: %f" %(recall_score(y_test, y_pred, average="macro")))
print("Accuracy...: %f" %(accuracy_score(y_test, y_pred)))

F1.........: 0.698964
Precision..: 0.717400
Recall.....: 0.715865
Accuracy...: 0.699000


In [15]:
import sklearn.metrics as metrics

print(metrics.classification_report(y_test, y_pred, digits=3))

              precision    recall  f1-score   support

           0      0.800     0.623     0.701       573
           1      0.610     0.792     0.689       427

   micro avg      0.695     0.695     0.695      1000
   macro avg      0.705     0.707     0.695      1000
weighted avg      0.719     0.695     0.696      1000



In [22]:
x1 = preprocessing("i am going to build a wall for the immigrants")
x1 = [x1]
y1 = tokenizer.texts_to_sequences(x1)
tweets1 = sequence.pad_sequences(y1, maxlen=maxlen)
model.predict(tweets1)

array([[0.9214898]], dtype=float32)